In [1]:
# Jupyter relative import (else it interupts the module)
import os, sys
parent_dir = os.path.abspath('..')
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
    
from cassandra.auth import PlainTextAuthProvider
from cassandra.cluster import Cluster
from cassandra.cqlengine import connection, management, query
from cassandra.cqlengine.connection import log as cql_logger
from config import get_settings
from db import create_session
import pandas as pd

**NoSQL Databases**
- Impossible to perform joins
- Cannot simply 'filter' tables - instead the column needs to be extracts to a key and wildcarded
- Denormalized so you only have to query one table for the data (even if its duplicated)
- Design the tables base on the query results you expect (happens in SQL as they become less performant)
- If a table isn’t designed for a query:
    - Not possible to run the query
    - Extremely slow/unperformant (down regulate availability)

Even if we create a foreign key on the Employee table for a car_make id, the car make isn't unique per employee, so we lose the ability to query the 01 notebook tables for an employees car make.  Instead we need to extend the data model to allow us to query an employees car make from one table.


```By using a composite key, we can partition all the car makes in a new table, then include a clustering column to remove the need for a join (now we use a WHERE) that will return the car make for an employee_id```

<br>

**Table**
employee_by_car_make

**Columns**
car_make, employee_id, firstname, surname, age

**Composite key of (car_make, employee_id)** where:

    car_make = partition key (tells us which node)
    id = clustering column (defines the order of the data)

In [2]:
session = create_session()

# Create a keyspace
session.execute("""
CREATE KEYSPACE IF NOT EXISTS cql_keyspace
WITH replication = { 'class': 'SimpleStrategy', 'replication_factor': 1 } 
AND durable_writes = 'true';
""")

# Create tables in cql_keyspace for employee_by_car_make
session.execute("CREATE TABLE IF NOT EXISTS cql_keyspace.employee_by_car_make (car_make text, employee_id int, firstname text, surname text, age int, PRIMARY KEY(car_make, employee_id))")

# Insert a record into each keyspace table
session.execute("INSERT INTO cql_keyspace.employee_by_car_make (car_make, employee_id, firstname, surname, age) VALUES ('Toyota', 1, 'John', 'Doe', 26)")
session.execute("INSERT INTO cql_keyspace.employee_by_car_make (car_make, employee_id, firstname, surname, age) VALUES ('Mazda', 2, 'Jane', 'Doe', 26)")
session.execute("INSERT INTO cql_keyspace.employee_by_car_make (car_make, employee_id, firstname, surname, age) VALUES ('Toyota', 3, 'Emily', 'Smith', 26)")
session.execute("INSERT INTO cql_keyspace.employee_by_car_make (car_make, employee_id, firstname, surname, age) VALUES ('Mazda', 4, 'Matthew', 'Brown', 26)")

Since the partition key is `car_make`, they will be grouped/ordered on this column first, then clustered by the `employee_id` afterward.

This example could theoretically scale to `2 nodes`, where our hex function divides each of the two car_make across seperate nodes.

In [3]:
# Execute a CQL query to retrieve the record from the employee_by_car_make table
result = session.execute("SELECT * FROM cql_keyspace.employee_by_car_make")

# Convert the result to a Pandas DataFrame
pd.DataFrame(list(result)).head()

,car_make,employee_id,age,firstname,surname
0,Toyota,1,26,John,Doe
1,Toyota,3,26,Emily,Smith
2,Mazda,2,26,Jane,Doe
3,Mazda,4,26,Matthew,Brown


To query which employee drives a Toyota, I can now directly query this table with a WHERE clause of Toyota. Alternatively, if I want to find what car John drives, I'll need to have his employee_id available.

In [4]:
result = session.execute("SELECT * FROM cql_keyspace.employee_by_car_make WHERE car_make = 'Toyota'")
pd.DataFrame(list(result)).head()

,car_make,employee_id,age,firstname,surname
0,Toyota,1,26,John,Doe
1,Toyota,3,26,Emily,Smith


**Scenario:**

Since Toyota is such a common brand of car, and we originally dedicated our partioning key on `car_make`, we may find that the node which contains the Toyota `partition starts running very hot`.

To solve this we can use multiple columns to make up our `partition key` and one `clustering key`.

In [5]:
session = create_session()

session.execute("CREATE TABLE IF NOT EXISTS cql_keyspace.employee_by_car_make_and_model (car_make text, car_model text, employee_id int, firstname text, surname text, age int, PRIMARY KEY((car_make, car_model), employee_id))")
session.execute("INSERT INTO cql_keyspace.employee_by_car_make_and_model (car_make, car_model, employee_id, firstname, surname, age) VALUES ('Toyota', 'Hilux', 1, 'John', 'Doe', 26)")
session.execute("INSERT INTO cql_keyspace.employee_by_car_make_and_model (car_make, car_model, employee_id, firstname, surname, age) VALUES ('Mazda', 'CX5', 2, 'Jane', 'Doe', 26)")
session.execute("INSERT INTO cql_keyspace.employee_by_car_make_and_model (car_make, car_model, employee_id, firstname, surname, age) VALUES ('Toyota', 'Corolla', 3, 'Emily', 'Smith', 26)")
session.execute("INSERT INTO cql_keyspace.employee_by_car_make_and_model (car_make, car_model, employee_id, firstname, surname, age) VALUES ('Mazda', 'CX7', 4, 'Matthew', 'Brown', 26)")


In [6]:
result = session.execute("SELECT * FROM cql_keyspace.employee_by_car_make_and_model")
pd.DataFrame(list(result)).head()

,car_make,car_model,employee_id,age,firstname,surname
0,Mazda,CX7,4,26,Matthew,Brown
1,Toyota,Corolla,3,26,Emily,Smith
2,Mazda,CX5,2,26,Jane,Doe
3,Toyota,Hilux,1,26,John,Doe


In [7]:
result = session.execute("SELECT * FROM cql_keyspace.employee_by_car_make_and_model WHERE car_make = 'Toyota' AND car_model = 'Hilux'")
pd.DataFrame(list(result)).head()

,car_make,car_model,employee_id,age,firstname,surname
0,Toyota,Hilux,1,26,John,Doe
